### Задание № 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
select count(book_id)
from books
where publication_date::date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


### Задание № 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''
with reviews as
(
select count(review_id) as re_cnt, book_id
from reviews 
group by book_id), 

rating as 
(
select avg(rating), book_id
from ratings
group by book_id)

select ra.book_id, avg, re_cnt
from rating as ra
left join reviews as re on ra.book_id = re.book_id 
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,avg,re_cnt
0,1,3.666667,2.0
1,2,2.500000,1.0
2,3,4.666667,3.0
3,4,4.500000,2.0
4,5,4.000000,4.0
...,...,...,...
995,996,3.666667,3.0
996,997,3.400000,3.0
997,998,3.200000,4.0
998,999,4.500000,2.0


### Задание № 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [9]:
query = '''
select p.publisher, count(distinct b.book_id)
from books as b
join publishers as p on p.publisher_id = b.publisher_id
where num_pages > 50
group by p.publisher
order by count(distinct b.book_id) desc
limit 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


### Задание № 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query = '''
/*находим книги с более 50 оценками (book_id), присоединяем авторов к таблице (authors)/
, делим сумму всех оценок на количество оценок, чтобы узнать avg*/

with books_id as 
(
select book_id,
       count(rating_id) as cnt_rating
       , sum(rating) as sum_rating
from ratings
group by book_id
having count(rating_id)>50
),

authors as
(
select books_id.book_id, books_id.cnt_rating, a.author, sum_rating
from books_id
join books as b on books_id.book_id = b.book_id
join authors as a on a.author_id = b.author_id)

select author, sum(sum_rating)/sum(cnt_rating) as avg_rating
from authors
group by author
order by avg_rating desc
'''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


### Задача № 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:

query = '''
/*делаем 2 СТЕ. 1 находим айди пользователей (больше 50 оценок). 2 считаем количество обзоров по каждому пользователю*/ 

with username as
(select distinct username
from ratings
group by username
having count(username) > 50), 

cnt_rating as 
(select  count(reviews.review_id), reviews.username
from reviews
join username on username.username = reviews.username
group by reviews.username)

select avg(count)
from cnt_rating
'''

pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333
